# Settings

## Installation

In [1]:
!pip install --upgrade autocarver

## Setting up samples

This dataset can be found from the corresponding Kaggle competition at https://www.kaggle.com/competitions/GiveMeSomeCredit/

In [1]:
import pandas as pd

data_path = "GiveMeSomeCredit"

credit_data = pd.read_csv(f"{data_path}/cs-training.csv", index_col=0)
print(credit_data.shape)
credit_data.head()

(150000, 11)


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_dev = train_test_split(credit_data, test_size=0.33, random_state=42)

## Picking up columns to Carve

In [3]:
X_train.dtypes

SeriousDlqin2yrs                          int64
RevolvingUtilizationOfUnsecuredLines    float64
age                                       int64
NumberOfTime30-59DaysPastDueNotWorse      int64
DebtRatio                               float64
MonthlyIncome                           float64
NumberOfOpenCreditLinesAndLoans           int64
NumberOfTimes90DaysLate                   int64
NumberRealEstateLoansOrLines              int64
NumberOfTime60-89DaysPastDueNotWorse      int64
NumberOfDependents                      float64
dtype: object

In [4]:
X_train.isna().mean()

SeriousDlqin2yrs                        0.000000
RevolvingUtilizationOfUnsecuredLines    0.000000
age                                     0.000000
NumberOfTime30-59DaysPastDueNotWorse    0.000000
DebtRatio                               0.000000
MonthlyIncome                           0.197383
NumberOfOpenCreditLinesAndLoans         0.000000
NumberOfTimes90DaysLate                 0.000000
NumberRealEstateLoansOrLines            0.000000
NumberOfTime60-89DaysPastDueNotWorse    0.000000
NumberOfDependents                      0.026129
dtype: float64

In [5]:
target = "SeriousDlqin2yrs"
quantitative_features = [feature for feature in X_train if feature != target]

# Feature processing with AutoCarver
## Fitting train samples and testint robustness

In [6]:
from AutoCarver import AutoCarver

auto_carver = AutoCarver(
    quantitative_features=quantitative_features,
    qualitative_features=[],
    sort_by='cramerv',  # Best combination according to Cramer's V
    dropna=False,  # don't want to groups nans with other values, leave that to XGBoost 
    min_freq=0.1,  # minimum frequency per modality
    max_n_mod=5,  # maximum number of modality per carved feature
    copy=True,  # in order not to modify X_train directly
    pretty_print=True,  # prints nice tables
)
x_discretized = auto_carver.fit_transform(
    # specifying dataset to carve
    X_train, X_train[target],
    # specifying a dataset to test robustness
    X_dev=X_dev, y_dev=X_dev[target]
)

------
[Discretizer] Fit Quantitative Features
---
 - [QuantileDiscretizer] Fit ['age', 'NumberOfDependents', 'DebtRatio', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTimes90DaysLate', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'MonthlyIncome']
 - [BaseDiscretizer] Transform Quantitative ['age', 'NumberOfDependents', 'DebtRatio', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTimes90DaysLate', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'MonthlyIncome']
 - [OrdinalDiscretizer] Fit ['NumberOfDependents', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTimes90DaysLate', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'age', 'MonthlyIncome']
------

 - [BaseDiscretize

,target_rate,frequency
NumberOfDependents,,
x <= 0.0,0.060000,0.580000
0.0 < x <= 1.0,0.073000,0.175000
1.0 < x,0.085000,0.219000
__NAN__,0.048000,0.026000
,target_rate,frequency
NumberOfDependents,,
x <= 0.0,0.057000,0.579000
0.0 < x <= 1.0,0.074000,0.177000
1.0 < x,0.086000,0.218000


Testing robustness    :   0%|          | 0/3 [00:00<?, ?it/s]


 - [AutoCarver] Carved feature distribution


,target_rate,frequency
x <= 0.0,0.060000,0.580000
0.0 < x <= 1.0,0.073000,0.175000
1.0 < x,0.085000,0.219000
__NAN__,0.048000,0.026000
,target_rate,frequency
x <= 0.0,0.057000,0.579000
0.0 < x <= 1.0,0.074000,0.177000
1.0 < x,0.086000,0.218000
__NAN__,0.042000,0.026000


...

,target_rate,frequency
MonthlyIncome,,
x <= 2.3K,0.085000,0.101000
2.3K < x <= 3.4K,0.097000,0.101000
3.4K < x <= 5.4K,0.080000,0.201000
5.4K < x <= 8.2K,0.061000,0.199000
8.2K < x <= 10.7K,0.051000,0.100000
10.7K < x,0.044000,0.100000
__NAN__,0.056000,0.197000
,target_rate,frequency
MonthlyIncome,,


Testing robustness    :   0%|          | 0/30 [00:00<?, ?it/s]


 - [AutoCarver] Carved feature distribution


,target_rate,frequency
x <= 2.3K,0.085000,0.101000
2.3K < x <= 3.4K,0.097000,0.101000
3.4K < x <= 5.4K,0.080000,0.201000
5.4K < x <= 8.2K,0.061000,0.199000
8.2K < x <= 10.7K,0.047000,0.201000
__NAN__,0.056000,0.197000
,target_rate,frequency
x <= 2.3K,0.089000,0.101000
2.3K < x <= 3.4K,0.098000,0.101000
3.4K < x <= 5.4K,0.075000,0.199000


------

 - [BaseDiscretizer] Transform Quantitative ['age', 'NumberOfDependents', 'DebtRatio', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'MonthlyIncome']


## Inspecting Discretization

In [7]:
x_discretized[quantitative_features].head()

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
87936,0.0,4,0,4.0,NaN,2,0,1,0,0.0
3893,3.0,0,0,2.0,2.0,1,0,0,0,0.0
41405,4.0,1,0,4.0,NaN,2,0,1,0,0.0
91125,2.0,2,0,2.0,3.0,3,0,0,0,2.0
67373,3.0,2,3,4.0,NaN,2,2,1,1,NaN


In [7]:
auto_carver.summary()

label                 content
feature                              dtype                               
DebtRatio                            float      0           [x <= 287.6m]
                                     float      1  [287.6m < x <= 467.4m]
                                     float      2  [467.4m < x <= 648.0m]
                                     float      3     [648.0m < x <= 3.8]
                                     float      4               [3.8 < x]
MonthlyIncome                        float      0             [x <= 2.3K]
                                     float      1      [2.3K < x <= 3.4K]
                                     float      2      [3.4K < x <= 5.4K]
                                     float      3      [5.4K < x <= 8.2K]
                                     float      4              [8.2K < x]
                                     float      5               [__NAN__]
NumberOfDependents                   float      0              [x <= 0.0]
                                     float      1        [0.0 < x <= 1.0]
                                     float      2               [1.0 < x]
                                     float      3               [__NAN__]
NumberOfOpenCreditLinesAndLoans      float      0              [x <= 3.0]
                                     float      1        [3.0 < x <= 5.0]
                                     float      2        [5.0 < x <= 8.0]
                                     float      3       [8.0 < x <= 12.0]
                                     float      4              [12.0 < x]
NumberOfTime30-59DaysPastDueNotWorse float      0              [x <= 0.0]
                                     float      1               [0.0 < x]
NumberRealEstateLoansOrLines         float      0              [x <= 0.0]
                                     float      1        [0.0 < x <= 1.0]
                                     float      2               [1.0 < x]
RevolvingUtilizationOfUnsecuredLines float      0           [x <= 155.4m]
                                     float      1  [155.4m < x <= 273.6m]
                                     float      2  [273.6m < x <= 447.4m]
                                     float      3  [447.4m < x <= 700.7m]
                                     float      4            [700.7m < x]
age                                  float      0             [x <= 33.0]
                                     float      1      [33.0 < x <= 48.0]
                                     float      2      [48.0 < x <= 56.0]
                                     float      3      [56.0 < x <= 61.0]
                                     float      4              [61.0 < x]

## Saving for later uses

In [9]:
import json

# storing as json file
with open('my_carver.json', 'w') as my_carver_json:
    json.dump(auto_carver.to_json(), my_carver_json)

# Feature Selection

### Setting up measures and filters

In [10]:
from AutoCarver.feature_selection import FeatureSelector

n_best = 10  # number of features to select

feature_selector = FeatureSelector(
    quantitative_features=quantitative_features, 
    n_best=n_best,
    pretty_print=True,
)
best_features = feature_selector.select(x_discretized, x_discretized[target])

------
[FeatureSelector] Selecting from Features: ['age', 'NumberOfDependents', 'DebtRatio', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTimes90DaysLate', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'MonthlyIncome']
---

 - Association between X and y


,dtype,pct_nan,pct_mode,mode,kruskal_measure
NumberOfTimes90DaysLate,int64,0.000000,0.943960,0,11854.250713
NumberOfTime60-89DaysPastDueNotWorse,int64,0.000000,0.949254,0,7636.377475
RevolvingUtilizationOfUnsecuredLines,float64,0.000000,0.500000,0.000000,6158.299923
NumberOfTime30-59DaysPastDueNotWorse,int64,0.000000,0.839035,0,6076.412797
age,int64,0.000000,0.303592,1,1370.032527
MonthlyIncome,float64,0.197383,0.200945,2.000000,337.382149
NumberOfDependents,float64,0.026129,0.579741,0.000000,175.384642
NumberOfOpenCreditLinesAndLoans,int64,0.000000,0.261602,2,121.575254
NumberRealEstateLoansOrLines,int64,0.000000,0.375512,0,120.839911
DebtRatio,float64,0.000000,0.400000,0.000000,47.123007



 - Association between X and y, filtered for inter-feature assocation


,dtype,pct_nan,pct_mode,mode,kruskal_measure
NumberOfTimes90DaysLate,int64,0.000000,0.943960,0,11854.250713
NumberOfTime60-89DaysPastDueNotWorse,int64,0.000000,0.949254,0,7636.377475
RevolvingUtilizationOfUnsecuredLines,float64,0.000000,0.500000,0.000000,6158.299923
NumberOfTime30-59DaysPastDueNotWorse,int64,0.000000,0.839035,0,6076.412797
age,int64,0.000000,0.303592,1,1370.032527
MonthlyIncome,float64,0.197383,0.200945,2.000000,337.382149
NumberOfDependents,float64,0.026129,0.579741,0.000000,175.384642
NumberOfOpenCreditLinesAndLoans,int64,0.000000,0.261602,2,121.575254
NumberRealEstateLoansOrLines,int64,0.000000,0.375512,0,120.839911
DebtRatio,float64,0.000000,0.400000,0.000000,47.123007


------



['NumberOfTimes90DaysLate',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'RevolvingUtilizationOfUnsecuredLines',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'age',
 'MonthlyIncome',
 'NumberOfDependents',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberRealEstateLoansOrLines',
 'DebtRatio']

### Enjoy modeling!